In [2]:
!pip install geopy

In [3]:
!pip install --upgrade pip

In [4]:
import requests
import urllib.parse as urlparse
from geopy import distance

In [5]:
naver_KEY_ID = 'nWxD0TpBM0o2UmWor2YT'
naver_KEY_secret = 'yNSdVVOHuN'

In [6]:
class Route:
    def __init__(self, headers={"X-NCP-APIGW-API-KEY-ID": naver_KEY_ID,
           "X-NCP-APIGW-API-KEY": naver_KEY_secret}):
        self.headers = headers
    
    # 각 출발지 -> 위경도 변환
    
    def addr_to_xy(self):
        # 주소값 입력
        self.d1_name = input("출발지 1을 입력하세요. : ")
        self.d2_name = input("출발지 2를 입력하세요. : ")
        # URL 설정
        self.d1_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d1_name)
        self.d2_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d2_name)
        # 리퀘스트
        self.response1 = requests.get(self.d1_url, headers=self.headers)
        self.response2 = requests.get(self.d2_url, headers=self.headers)
        # JSON 파싱하여 위경도 추출
        self.d1_x = self.response1.json()["addresses"][0]["x"] 
        self.d1_y = self.response1.json()["addresses"][0]["y"]
        self.d2_x = self.response2.json()["addresses"][0]["x"]
        self.d2_y = self.response2.json()["addresses"][0]["y"]
        return self.d1_x, self.d1_y, self.d2_x, self.d2_y
    
    # 출발지간 위경도 -> 경로값 변환
    def road_path(self):
        self.d1_xy = str(self.d1_x) + "," + str(self.d1_y)
        self.d2_xy = str(self.d2_x) + "," + str(self.d2_y)
        self.d_option = "traoptimal"
        self.d_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={}&goal={}&option={}".format(self.d1_xy, self.d2_xy, self.d_option)
        self.response = requests.get(self.d_url, headers=self.headers)
        self.path = self.response.json()["route"]["traoptimal"][0]["path"]
        self.avg_path = self.response.json()["route"]["traoptimal"][0]["summary"]["distance"] / len(self.path)
        print("거리 : {}m, ".format(self.response.json()["route"]["traoptimal"][0]["summary"]["distance"]), len(self.path), ", ", self.avg_path)
        return self.path
    
    # 출발지간 대중교통 경로
    def trans_path(self, path_type=0):
        self.path_type = path_type
        self.key = 'ODSAY KEY'
        self.url = 'https://api.odsay.com/v1/api/searchPubTransPathT'
        self.params = {'apiKey' : self.key,
                  'SX' : self.d1_x,
                  'SY' : self.d1_y,
                  'EX' : self.d2_x,
                  'EY' : self.d2_y,
                  'SearchPathType' : self.path_type,
                  'OPT':0,
                 }
        self.url = self.url + '?' + urlparse.urlencode(self.params)
        self.response = requests.get(self.url)
        # 경로내 정류장 위경도 추출
        self.transit_count = self.response.json()['result']['path'][0]['info']['busTransitCount'] + self.response.json()['result']['path'][0]['info']['subwayTransitCount']
        self.distance = round(self.response.json()['result']['path'][0]['info']['totalDistance'] / 1000,2)
        self.stop_lat_lng = []
        for i in range(1, 2*self.transit_count, 2):
            self.stop_info = self.response.json()['result']['path'][0]['subPath'][i]['passStopList']['stations']
            self.stop_lat_lng += [(float(a['y']),float(a['x'])) for a in self.stop_info]
        # 경로가 1km이하거나, 정류장이 5개 미만이면 모든 위경도 표출하고, 이외에는 중간 6개의 정류장 위경도만 표출
        if self.distance <= 1 or len(self.stop_lat_lng) <= 5:
            self.meet_point = self.stop_lat_lng
        else:
            self.meet_point = self.stop_lat_lng[len(self.stop_lat_lng)//2-3:len(self.stop_lat_lng)//2+3]
        print('거리: {}km'.format(self.distance))
        return self.meet_point, self.stop_lat_lng


In [7]:
rte = Route()


In [24]:
d1_x, d1_y, d2_x, d2_y = rte.addr_to_xy()
d1_x, d1_y, d2_x, d2_y


출발지 1을 입력하세요. : 서울특별시 종로구 당주동 128-14
출발지 2를 입력하세요. : 서울특별시 서대문구 냉천동 183-2


KeyError: 'addresses'

In [11]:
# 대중교통 이동
meet_points, all_points = rte.trans_path()


AttributeError: 'Route' object has no attribute 'd1_x'

In [12]:
# 경로내 위, 경도 최대 최소값 구하기
lat_list = [point[0] for point in all_points]
lng_list = [point[1] for point in all_points]


NameError: name 'all_points' is not defined

In [13]:
min(lat_list), max(lat_list), min(lng_list), max(lng_list)


NameError: name 'lat_list' is not defined

In [14]:
df = pd.read_csv('mango_r.csv')
df.info()


FileNotFoundError: [Errno 2] No such file or directory: 'mango_r.csv'

In [15]:
df['latlng'] = df.apply(lambda x:(x['lat'], x['lng']), axis=1)
df.tail()


NameError: name 'df' is not defined

In [16]:
# 경로 square 내 맛집 추출
direction_square = df[(df['lat']<=max(lat_list)) & (df['lat']>=min(lat_list)) & (df['lng']>=min(lng_list)) & (df['lng']<=max(lng_list))]


NameError: name 'df' is not defined

In [17]:
# square 내 경로 1km 이내 맛집 추출
matzip=pd.DataFrame()
for lat_lng in direction_square['latlng']:
    for point in meet_points:
        dis = distance.distance(point, lat_lng).km
        if dis <= 1:
            df_1 = direction_square[direction_square['latlng'] == lat_lng]
            matzip = matzip.append(df_1).reset_index(drop=True)


NameError: name 'direction_square' is not defined

In [18]:
matzip


""


In [20]:
df = pd.read_csv('./df.csv')
df = df.drop(columns='Unnamed: 0')

In [23]:
cd0 = df.category == '한식'
cd1 = df.lat < 37.572380
cd2 = df.lat > 37.562380
cd3 = df.lng < 126.974200
cd4 = df.lng > 126.964200
cd_all = cd0 & cd1 & cd2 & cd3 & cd4
df[cd_all].sort_values(by='rating', ascending=False).head()

,fname,name,tel,addr,si,gu,dong,bizhour,category,menu,...,microreview,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating
45,광화문뚝감,광화문뚝감,02-722-5894,서울특별시 종로구 당주동 128-14,서울특별시,종로구,당주동,"매일 00:00~24:00 명절 휴무, 15:00~17:00브레이크타임",한식,"감자탕 25,000~41,000 | 해물뼈찜 29,000~44,000 | 뚝감(뚝배...",...,"['광화문 맛집 기다린 자만이 먹을 수 있는 뚝배기 감자탕', '그릇 뚝딱하는 뚝배...",https://ldb-phinf.pstatic.net/20190930_207/156...,126.972592,37.572069,4.45,3.53,4.30,3.05,3.54,4.18
5808,동해관,동해관,02-363-4221,서울특별시 서대문구 냉천동 183-2,서울특별시,서대문구,냉천동,평일 11:30~22:30,한식,"점심정식코스 25,000 | 저녁진미코스 40,000",...,[],https://ldb-phinf.pstatic.net/20200402_20/1585...,126.964985,37.566476,4.62,3.53,3.68,1.94,3.54,4.03
745,덕수정,덕수정,02-755-0180,서울특별시 중구 정동 11-5,서울특별시,중구,정동,매일 11:00~22:00 | 일요일 휴무,한식,"부대찌개 8,000 | 오징어볶음 10,000 | 삼치구이 8,000 | 우거지갈비...",...,"['오징어볶음이 맛있는 부대찌개전문점', '오징어볶음 고등학교 나를 살찌운 음식',...",https://ldb-phinf.pstatic.net/20150901_108/144...,126.972550,37.565900,4.33,3.53,4.00,2.75,3.54,4.02
5842,이판저판,이판저판,02-313-8629,서울특별시 서대문구 냉천동 227,서울특별시,서대문구,냉천동,매일 00:00~24:00,한식,"생삼겹살 9,000 | 막걸리 4,000 | 한라산 5,000 | 진한복분자 15,...",...,[],https://ldb-phinf.pstatic.net/20200302_107/158...,126.964462,37.567742,4.61,3.53,3.68,1.94,3.54,4.02
13,암소서울,암소서울,02-6744-0123,서울특별시 종로구 신문로1가 115 콘코디언빌딩 지하 1층,서울특별시,종로구,신문로1가,매일 11:00~21:00 | 매일 15:00~16:00 브레이크 타임,한식,"종가 159,000 | 반가 ( 런치 ) 79,000 | 회식 ( 6인이상 ) 99...",...,"['토니정쉐프의 한식다이닝을 맛볼 수 있는 광화문맛집', '광화문 맛집 이름걸고 하...",https://ldb-phinf.pstatic.net/20191129_184/157...,126.973197,37.569784,4.65,3.53,3.68,0.95,3.54,3.97


In [9]:
var map = new naver.maps.Map('map', {
    center: new naver.maps.LatLng(37.3595704, 127.105399),
    zoom: 15
});

var marker = new naver.maps.Marker({
    position: new naver.maps.LatLng(37.3595704, 127.105399),
    map: map
});

SyntaxError: invalid syntax (<ipython-input-9-05ee77621b94>, line 1)

In [10]:
<script type="text/javascript" src="https://openapi.map.naver.com/openapi/v3/maps.js?ncpClientId=YOUR_CLIENT_ID"></script>

SyntaxError: invalid syntax (<ipython-input-10-a03d0a6869f0>, line 1)

In [4]:
from flask import Flask
from flask_googlemaps import GoogleMaps

app = Flask(__name__)

# you can set key as config
app.config['GOOGLEMAPS_KEY'] = "8JZ7i18MjFuM35dJHq70n3Hx4"

# Initialize the extension
GoogleMaps(app)

# you can also pass the key here if you prefer
GoogleMaps(app, key="8JZ7i18MjFuM35dJHq70n3Hx4")


AssertionError: A name collision occurred between blueprints <flask.blueprints.Blueprint object at 0x000001BEA7E50808> and <flask.blueprints.Blueprint object at 0x000001BEA7E506C8>. Both share the same name "googlemaps". Blueprints that are created on the fly need unique names.

In [5]:
from flask import Flask, render_template
from flask_googlemaps import GoogleMaps
from flask_googlemaps import Map

app = Flask(__name__, template_folder=".")
GoogleMaps(app)

@app.route("/")
def mapview():
    # creating a map in the view
    mymap = Map(
        identifier="view-side",
        lat=37.4419,
        lng=-122.1419,
        markers=[(37.4419, -122.1419)]
    )
    sndmap = Map(
        identifier="sndmap",
        lat=37.4419,
        lng=-122.1419,
        markers=[
          {
             'icon': 'http://maps.google.com/mapfiles/ms/icons/green-dot.png',
             'lat': 37.4419,
             'lng': -122.1419,
             'infobox': "<b>Hello World</b>"
          },
          {
             'icon': 'http://maps.google.com/mapfiles/ms/icons/blue-dot.png',
             'lat': 37.4300,
             'lng': -122.1400,
             'infobox': "<b>Hello World from other place</b>"
          }
        ]
    )
    return render_template('example.html', mymap=mymap, sndmap=sndmap)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
<!DOCTYPE html>
<html>
  <head>
    {{"decoupled-map"|googlemap_js(37.4419, -122.1419, markers=[(37.4419,
    -122.1419)])}} {{mymap.js}} {{sndmap.js}}
  </head>
  <body>
    <h1>Flask Google Maps Example</h1>

    <h2>Template function centered, no marker</h2>
    {{googlemap("simple-map", 37.4419, -122.1419)}}

    <h2>Template filter decoupled with single marker</h2>
    {{"decoupled-map"|googlemap_html(37.4419, -122.1419)}}

    <h2>Template function with multiple markers</h2>
    {% with map=googlemap_obj("another-map", 37.4419, -122.1419,
    markers=[(37.4419, -122.1419), (37.4300, -122.1400)]) %} {{map.html}}
    {{map.js}} {% endwith %}

    <h2>First map generated in view</h2>
    {{mymap.html}}

    <h2>Second map generated in view</h2>
    <h3>Example for different icons in multiple markers with infoboxes</h3>
    {{sndmap.html}}
  </body>
</html>


SyntaxError: invalid syntax (<ipython-input-6-553e1de4a8d3>, line 1)

In [7]:
    Map(
        identifier="catsmap",
        lat=37.4419,
        lng=-122.1419,
        markers=[
            {
                'icon': 'http://maps.google.com/mapfiles/ms/icons/green-dot.png',
                'lat':  37.4419,
                'lng':  -122.1419,
                'infobox': "<img src='cat1.jpg' />"
            },
            {
                'icon': 'http://maps.google.com/mapfiles/ms/icons/blue-dot.png',
                'lat': 37.4300,
                'lng': -122.1400,
                'infobox': "<img src='cat2.jpg' />"
            },
            {
                'icon': 'http://maps.google.com/mapfiles/ms/icons/yellow-dot.png',
                'lat': 37.4500,
                'lng': -122.1350,
                'infobox': "<img src='cat3.jpg' />"
            }
        ]
    )


In [8]:
@app.route('/map-unbounded/')
def map_unbounded():
"""Create map with markers out of bounds."""
    locations = []    # long list of coordinates
    map = Map(
        lat=locations[0].latitude,
        lng=locations[0].longitude,
        markers=[(loc.latitude, loc.longitude) for loc in locations]
    )
    return render_template('map.html', map=map)


IndentationError: expected an indented block (<ipython-input-8-b7aa70ea3fc3>, line 3)